In [1]:
# Install python dependencies
%pip install torch transformers huggingface_hub omegaconf datasets==2.16.1 tqdm 
# Optinal python packages for better user experience
%pip install ipywidgets nbconvert
# Install conda dependencies
%conda install -c conda-forge -c pytorch -c nvidia faiss-gpu=1.11.0 

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Channels:
 - conda-forge
 - pytorch
 - nvidia
 - defaults
Platform: linux-64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 25.1.1
    latest version: 25.5.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import necessary libraries
import torch
import omegaconf
import collections
import os
import re
import numpy as np
import faiss
import gc
from tqdm import tqdm
from typing import Any
from collections import OrderedDict
from transformers import DPRQuestionEncoder, DPRContextEncoder, AutoTokenizer, DPRConfig, GPT2TokenizerFast
from huggingface_hub import hf_hub_download
from datasets import load_dataset, Dataset, DatasetDict

# Setup external services authentication
HF_TOKEN = os.getenv('HF_TOKEN')

In [3]:
def rename_keys_substring(ordered_dict: OrderedDict[str, Any], find_pattern, replace_pattern):
    """
    Rename keys in an OrderedDict by replacing substring occurrences using regular expressions.
    
    Args:
        ordered_dict: The OrderedDict to modify
        find_pattern: The regex pattern to find in keys
        replace_pattern: The replacement pattern (can include backreferences like \\1, \\2)
    
    Returns:
        New Mapping with renamed keys
    """
    new_dict = OrderedDict[str, Any]()
    compiled_pattern = re.compile(find_pattern)
    
    for key, value in ordered_dict.items():
        if not compiled_pattern.search(key):
            continue
            
        new_key = compiled_pattern.sub(replace_pattern, key)
        new_dict[new_key] = value
    return new_dict


In [4]:
# Ensure that the necessary types are registered for safe deserialization
torch.serialization.add_safe_globals(
    [
        omegaconf.dictconfig.ContainerMetadata,
        omegaconf.dictconfig.DictConfig,
        omegaconf.base.Metadata,
        omegaconf.nodes.AnyNode,
        omegaconf.listconfig.ListConfig,
        collections.defaultdict,
        Any,
        dict,
        list,
        int,
    ]
)

# Check if CUDA is available and set device accordingly
device = "cpu" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load model from checkpoint
checkpoint_path = hf_hub_download(
    repo_id="NTU-NLP-sg/xCodeEval-nl-code-starencoder-ckpt-37",
    filename="dpr_biencoder.37.pt",
    repo_type="model",
    token=HF_TOKEN,
)
state_dict = torch.load(checkpoint_path, map_location=device)

# Retrieve fine-tuned weights
# Pattern matches: question_model.embeddings/encoder.* -> question_encoder.bert_model.*
ctx_state_dict = rename_keys_substring(
    state_dict["model_dict"],
    r"ctx_model\.(embeddings|encoder)\.([Ll]ayer|token|word|position_embeddings)",
    r"ctx_encoder.bert_model.\1.\2",
)

# Initialize encoders
pretrained_model_name = state_dict["encoder_params"]["encoder"]["pretrained_model_cfg"]
encoder_config = DPRConfig.from_pretrained(
    pretrained_model_name,
    token=HF_TOKEN,
)

ctx_encoder = DPRContextEncoder.from_pretrained(
    None, state_dict=ctx_state_dict, config=encoder_config, token=HF_TOKEN
)
ctx_encoder = ctx_encoder.to(device).eval()

# Distribute workload across multiple GPUs if available
# Not so efficient as DistributedDataParallel, but simpler for single-node setups
# DataParallel will split the input across the GPUs and gather the outputs
# This is useful for inference or when the model is not too large
if torch.cuda.device_count() > 1:
    ctx_encoder = torch.nn.DataParallel(ctx_encoder)
    print(f"Using {torch.cuda.device_count()} GPUs for context encoder")

# Initialize tokenizer
tokenizer: GPT2TokenizerFast = AutoTokenizer.from_pretrained(
    pretrained_model_name, config=encoder_config
)
tokenizer.pad_token = tokenizer.eos_token

Using device: cpu


You are using a model of type bert to instantiate a model of type dpr. This is not supported for all configurations of models and can yield errors.


In [5]:
def print_memory_usage():
    """Print current memory usage statistics"""
    if torch.cuda.is_available():
        torch.cuda.synchronize()  # Ensure all GPU operations are complete
        for i in range(torch.cuda.device_count()):
            allocated = torch.cuda.memory_allocated(i) / (1024**3)
            reserved = torch.cuda.memory_reserved(i) / (1024**3)
            total = torch.cuda.get_device_properties(i).total_memory / (1024**3)
            print(f"GPU {i}: {allocated:.2f}GB allocated, {reserved:.2f}GB reserved, {total:.2f}GB total")
    else:
        print("CUDA is not available. No GPU memory stats to report.")

def cleanup_memory():
    """Comprehensive memory cleanup"""
    # Clear Python garbage
    gc.collect()
    
    # Clear PyTorch cache
    torch.cuda.empty_cache()
    
    # Print memory stats
    print_memory_usage()

In [6]:
def embed_codes(batch: DatasetDict[str, list]):
    inputs = tokenizer(
        batch["source_code"],
        padding="max_length",
        truncation=True,
        max_length=1024,
        return_tensors="pt"  # Use PyTorch tensors 
    )
    # Move to device in one operation
    inputs = {k: v.to(device, non_blocking=True) for k, v in inputs.items()}
    
    with torch.no_grad(), torch.amp.autocast(device):
        print("Embedding batch...")
        print_memory_usage()
        embeddings = ctx_encoder(**inputs).pooler_output
        print("Batch embedded.")
        print_memory_usage()
        # Convert to float16 to save memory if precision allows
        embeddings_cpu = embeddings.cpu().numpy().astype(np.float32).tolist()
        return {"embedding": embeddings_cpu}
    

In [ ]:
# Load corpus dataset
corpus = load_dataset(
    "NTU-NLP-sg/xCodeEval",
    "retrieval_corpus",
    trust_remote_code=True,
    split="test",
    revision="467d25a839086383794b58055981221b82c0d107",
    token=HF_TOKEN,
)

print_memory_usage()

corpus = corpus.map(
    embed_codes,
    batched=True,
    batch_size=128,
)
i = 0

GPU 0: 3.89GB allocated, 17.68GB reserved, 19.57GB total
GPU 1: 0.00GB allocated, 0.00GB reserved, 19.57GB total


Map:   0%|          | 0/25043700 [00:00<?, ? examples/s]

Embedding batch...
GPU 0: 3.90GB allocated, 17.68GB reserved, 19.57GB total
GPU 1: 0.00GB allocated, 0.00GB reserved, 19.57GB total
Batch embedded.
GPU 0: 4.77GB allocated, 19.01GB reserved, 19.57GB total
GPU 1: 0.00GB allocated, 0.00GB reserved, 19.57GB total
Embedding batch...
GPU 0: 3.90GB allocated, 19.01GB reserved, 19.57GB total
GPU 1: 0.00GB allocated, 0.00GB reserved, 19.57GB total


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.35 GiB. GPU 0 has a total capacity of 19.57 GiB of which 282.00 MiB is free. Including non-PyTorch memory, this process has 19.29 GiB memory in use. Of the allocated memory 15.10 GiB is allocated by PyTorch, and 3.97 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)